In [ ]:
!pip install --quiet gradio opencv-python moviepy numpy mediapipe Pillow

In [ ]:
import os
import cv2
import numpy as np
import gradio as gr
import mediapipe as mp
from PIL import Image
from moviepy.editor import ImageClip, concatenate_videoclips, AudioFileClip
import traceback

mp_face_mesh = mp.solutions.face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1,
                                               refine_landmarks=True, min_detection_confidence=0.5)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def align_face(image_rgb, output_size=(640,640), eye_target=(320,240), ref_eye_dist=120):
    try:
        h, w, _ = image_rgb.shape
        img = image_rgb.astype(np.uint8)
        res = mp_face_mesh.process(img)
        left_eye = None
        right_eye = None
        if res and res.multi_face_landmarks:
            lm = res.multi_face_landmarks[0].landmark
            left_idxs = [33, 133]
            right_idxs = [362, 263]
            left_eye = np.mean([(lm[i].x * w, lm[i].y * h) for i in left_idxs], axis=0)
            right_eye = np.mean([(lm[i].x * w, lm[i].y * h) for i in right_idxs], axis=0)
        if left_eye is None or right_eye is None:
            gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
            faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
            if len(faces) == 0:
                return None
            x, y, wb, hb = max(faces, key=lambda f: f[2]*f[3])
            cx = x + wb/2
            cy = y + hb/3.0
            left_eye = np.array([cx - wb*0.18, cy])
            right_eye = np.array([cx + wb*0.18, cy])
        left_eye = np.array(left_eye, dtype=np.float32)
        right_eye = np.array(right_eye, dtype=np.float32)
        eyes_center = (left_eye + right_eye) / 2.0
        dx = right_eye[0] - left_eye[0]
        dy = right_eye[1] - left_eye[1]
        current_eye_dist = np.hypot(dx, dy)
        if current_eye_dist < 1.0:
            return None
        angle = np.degrees(np.arctan2(dy, dx))
        scale = ref_eye_dist / current_eye_dist
        cx, cy = eyes_center
        M = cv2.getRotationMatrix2D((cx, cy), angle, scale)
        eyes_center_rot = np.dot(M, np.array([cx, cy, 1.0]))
        tx = eye_target[0] - eyes_center_rot[0]
        ty = eye_target[1] - eyes_center_rot[1]
        M[0,2] += tx
        M[1,2] += ty
        out_w, out_h = output_size
        warped = cv2.warpAffine(img, M, (out_w, out_h), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT)
        return warped
    except Exception as e:
        print("Erreur align_face:", e)
        traceback.print_exc()
        return None

def create_face_movie(files, music=None, output_size=(640,640), eye_target=(320,240),
                      ref_eye_dist=120, duration_per_face=2.0):
    try:
        os.makedirs('outputs', exist_ok=True)
        clips = []
        processed = 0
        debug = []
        for f in files:
            try:
                pil = Image.open(f.name).convert('RGB')
                img = np.array(pil)
            except Exception as e:
                msg = f"Impossible de lire {getattr(f,'name',str(f))}: {e}"
                print(msg); debug.append(msg); continue
            aligned = align_face(img, output_size=output_size, eye_target=eye_target, ref_eye_dist=ref_eye_dist)
            if aligned is None:
                msg = f"Aucun visage alignable dans {os.path.basename(f.name)}"
                print(msg); debug.append(msg); continue
            clip = ImageClip(aligned).set_duration(duration_per_face)
            clips.append(clip)
            processed += 1
        if processed == 0:
            return ("Aucun visage aligné. Détails:\n" + "\n".join(debug), None, None)
        final = concatenate_videoclips(clips, method='compose')
        if music is not None:
            try:
                audio = AudioFileClip(music.name)
                audio = audio.set_duration(final.duration)
                final = final.set_audio(audio)
            except Exception as e:
                msg = f"Erreur audio (fichier ignoré): {e}"
                print(msg); debug.append(msg)
        out_path = os.path.join('outputs', 'face_movie_aligned.mov')
        final.write_videofile(out_path, fps=24, codec='libx264', audio_codec='aac',
                              ffmpeg_params=["-profile:v","baseline","-pix_fmt","yuv420p"])
        status = f"Vidéo générée : {processed} image(s)."
        if debug:
            status += "\n" + "\n".join(debug)
        return status, out_path, out_path
    except Exception as e:
        tb = traceback.format_exc()
        return (f"Erreur inattendue : {e}\n\nTraceback:\n{tb}", None, None)

iface = gr.Interface(
    fn=create_face_movie,
    inputs=[
        gr.File(label="Images (multiple)", file_types=["image"], file_count="multiple"),
        gr.File(label="Musique (optionnel)", file_types=["audio"])
    ],
    outputs=[
        gr.Textbox(label="Statut", lines=5),
        gr.Video(label="Lecture vidéo"),
        gr.File(label="Télécharger la vidéo")
    ],
    title="Face Movie — alignement fixe des yeux",
    description="Chaque photo est repositionnée/rotated/scalée pour que les yeux soient au même emplacement. Pas de zoom dynamique ni de transitions."
)

iface.launch(share=True)